## Wah-Wah Effect Filter Analysis
Comprehensive analysis, or a novel attempt, of the state-variable filter used in the wah-wah effect implementation.

In [1]:
# modules
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
import seaborn as sns
from matplotlib.gridspec import GridSpec
%matplotlib inline
import ipywidgets as widgets
from IPython.display import display
import pandas as pd
from matplotlib.colors import LinearSegmentedColormap

In [2]:
# plotting style
sns.set(style="whitegrid")
plt.rcParams["figure.figsize"] = (12, 6)

## Define Plotting Functions
Generate data and plot visualizations for the wah-wah effect filter.

In [3]:
def create_wah_filter(fs, f0, Q, filter_type='bandpass'):
    """
    Create a state-variable filter with specified parameters.
    
    Parameters:
    -----------
    fs : float
        Sampling frequency in Hz
    f0 : float
        Center frequency of the filter in Hz
    Q : float
        Quality factor of the filter
    filter_type : str
        Type of filter ('lowpass', 'highpass', 'bandpass')
        
    Returns:
    --------
    b, a : ndarray
        Filter coefficients
    """
    w0 = 2 * np.pi * f0 / fs  # Normalized frequency
    alpha = np.sin(w0) / (2 * Q)
    
    if filter_type == 'lowpass':
        b = [(1 - np.cos(w0))/2, 1 - np.cos(w0), (1 - np.cos(w0))/2]
        a = [1 + alpha, -2 * np.cos(w0), 1 - alpha]
    elif filter_type == 'highpass':
        b = [(1 + np.cos(w0))/2, -(1 + np.cos(w0)), (1 + np.cos(w0))/2]
        a = [1 + alpha, -2 * np.cos(w0), 1 - alpha]
    else:  # bandpass
        b = [alpha, 0, -alpha]
        a = [1 + alpha, -2 * np.cos(w0), 1 - alpha]
    
    return b, a

def plot_frequency_response(fs, f0, Q, filter_type='bandpass'):
    """
    Plot the frequency response of the wah filter.
    
    Parameters:
    -----------
    fs : float
        Sampling frequency in Hz
    f0 : float
        Center frequency of the filter in Hz
    Q : float
        Quality factor of the filter
    filter_type : str
        Type of filter ('lowpass', 'highpass', 'bandpass')
    """
    b, a = create_wah_filter(fs, f0, Q, filter_type)
    
    # Compute frequency response
    w, h = signal.freqz(b, a, worN=8000)
    
    # Convert to Hz and dB
    f = w * fs / (2 * np.pi)
    
    # Handle potential division by zero in dB calculation
    h_abs = np.abs(h)
    h_abs[h_abs == 0] = np.finfo(float).eps  # Replace zeros with smallest positive float
    db = 20 * np.log10(h_abs)
    
    # Handle NaN and Inf values
    db = np.nan_to_num(db, nan=-100, posinf=100, neginf=-100)
    
    # Plot
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8))
    
    ax1.plot(f, db, 'b')
    ax1.set_xscale('log')
    ax1.set_xlim([20, fs/2])
    ax1.set_ylim([-40, 10])
    ax1.grid(True)
    ax1.set_xlabel('Frequency [Hz]')
    ax1.set_ylabel('Gain [dB]')
    ax1.set_title(f'Frequency Response - {filter_type.capitalize()} ({f0} Hz, Q={Q})')
    
    # Phase response
    phase = np.unwrap(np.angle(h))
    ax2.plot(f, phase, 'g')
    ax2.set_xscale('log')
    ax2.set_xlim([20, fs/2])
    ax2.grid(True)
    ax2.set_xlabel('Frequency [Hz]')
    ax2.set_ylabel('Phase [rad]')
    ax2.set_title('Phase Response')
    
    plt.tight_layout()
    return fig

def plot_wah_sweep(fs=44100, f0_min=500, f0_max=2500, Q=5, n_steps=10):
    """
    Plot multiple frequency responses as the center frequency is swept.
    
    Parameters:
    -----------
    fs : float
        Sampling frequency in Hz
    f0_min : float
        Minimum center frequency in Hz
    f0_max : float
        Maximum center frequency in Hz
    Q : float
        Quality factor of the filter
    n_steps : int
        Number of steps between f0_min and f0_max
    """
    fig, ax = plt.subplots(figsize=(12, 6))
    
    # Create colormap for the sweep
    colors = plt.cm.viridis(np.linspace(0, 1, n_steps))
    
    # Frequency range for plotting
    w = np.logspace(np.log10(20), np.log10(fs/2), 1000)
    
    f0_values = np.linspace(f0_min, f0_max, n_steps)
    
    for i, f0 in enumerate(f0_values):
        b, a = create_wah_filter(fs, f0, Q)
        w_normalized = 2 * np.pi * w / fs
        _, h = signal.freqz(b, a, worN=w_normalized)
        
        # Handle potential division by zero
        h_abs = np.abs(h)
        h_abs[h_abs == 0] = np.finfo(float).eps
        db = 20 * np.log10(h_abs)
        db = np.nan_to_num(db, nan=-100, posinf=100, neginf=-100)
        
        ax.plot(w, db, color=colors[i], alpha=0.8, label=f'{f0:.1f} Hz')
    
    ax.set_xscale('log')
    ax.set_xlim([20, fs/2])
    ax.set_ylim([-40, 10])  # Fixed y-limits for sweep plot
    ax.set_xlabel('Frequency [Hz]')
    ax.set_ylabel('Gain [dB]')
    ax.set_title(f'Wah-Wah Filter Sweep (Q={Q})')
    ax.grid(True)
    
    # Custom legend with fewer entries
    legend_indices = np.linspace(0, n_steps-1, 5, dtype=int)
    handles, labels = ax.get_legend_handles_labels()
    ax.legend([handles[i] for i in legend_indices], [labels[i] for i in legend_indices], loc='upper right')
    
    plt.tight_layout()
    return fig

## Create Basic Plot
Basic visualization of the wah-wah filter response using our functions.

In [4]:
# Define parameters
fs = 44100  # Sampling frequency (Hz)
f0 = 1000   # Center frequency (Hz)
Q = 5       # Quality factor

# Create and display the basic frequency response plot
plt.figure(figsize=(12, 6))
basic_plot = plot_frequency_response(fs, f0, Q, 'bandpass')
plt.tight_layout()

## Format and Style Plot
Proper styling, titles, and customizations.

In [5]:
def create_styled_wah_plot(fs=44100, f0=1000, Q=5, filter_type='bandpass'):
    """Create a well-styled wah filter frequency response plot"""
    
    # Set seaborn style
    sns.set_style("darkgrid")
    
    # Create figure with GridSpec for flexible layout
    fig = plt.figure(figsize=(14, 8))
    gs = GridSpec(2, 2, height_ratios=[2, 1], width_ratios=[3, 1])
    
    # Main frequency response plot
    ax_main = fig.add_subplot(gs[0, 0])
    ax_phase = fig.add_subplot(gs[1, 0])
    ax_info = fig.add_subplot(gs[:, 1])
    
    # Get filter coefficients
    b, a = create_wah_filter(fs, f0, Q, filter_type)
    
    # Compute frequency response
    w, h = signal.freqz(b, a, worN=8000)
    f = w * fs / (2 * np.pi)
    
    # Handle potential division by zero in dB calculation
    h_abs = np.abs(h)
    h_abs[h_abs == 0] = np.finfo(float).eps  # Replace zeros with smallest positive float
    db = 20 * np.log10(h_abs)
    
    # Handle NaN and Inf values in dB
    db = np.nan_to_num(db, nan=-100, posinf=100, neginf=-100)
    
    phase = np.unwrap(np.angle(h))
    
    # Plot magnitude response
    ax_main.plot(f, db, color='dodgerblue', linewidth=2.5)
    ax_main.set_xscale('log')
    ax_main.set_xlim([20, fs/2])
    
    # Set fixed y-limits
    ax_main.set_ylim([-40, 10])
    
    ax_main.set_xlabel('Frequency [Hz]', fontsize=12)
    ax_main.set_ylabel('Gain [dB]', fontsize=12)
    ax_main.set_title(f'Wah-Wah Filter Frequency Response', 
                     fontsize=14, fontweight='bold')
    
    # Add grid with improved visibility
    ax_main.grid(True, which='major', linewidth=0.8, alpha=0.7)
    ax_main.grid(True, which='minor', linestyle=':', linewidth=0.5, alpha=0.5)
    
    # Highlight center frequency
    ax_main.axvline(f0, color='red', linestyle='--', alpha=0.7, label=f'Center freq: {f0} Hz')
    
    # Add annotations - only if we have valid data
    if np.isfinite(db).any():
        max_idx = np.argmax(db)
        if np.isfinite(db[max_idx]) and np.isfinite(f[max_idx]):
            ax_main.annotate(f'Peak: {db[max_idx]:.1f} dB @ {f[max_idx]:.0f} Hz',
                           xy=(f[max_idx], db[max_idx]),
                           xytext=(0.6*f[max_idx], db[max_idx]+3),
                           arrowprops=dict(arrowstyle='->'))
    
    # Phase response
    ax_phase.plot(f, phase, color='forestgreen', linewidth=2)
    ax_phase.set_xscale('log')
    ax_phase.set_xlim([20, fs/2])
    ax_phase.set_xlabel('Frequency [Hz]', fontsize=12)
    ax_phase.set_ylabel('Phase [rad]', fontsize=12)
    ax_phase.grid(True)
    
    # Information panel
    ax_info.axis('off')
    info_text = (
        f"Wah-Wah Filter Parameters\n"
        f"=======================\n\n"
        f"Filter Type: {filter_type.capitalize()}\n"
        f"Sampling Rate: {fs/1000:.1f} kHz\n"
        f"Center Frequency: {f0} Hz\n"
        f"Q Factor: {Q}\n\n"
        f"Filter Characteristics\n"
        f"----------------------\n"
        f"Bandwidth: {f0/Q:.1f} Hz\n"
    )
    ax_info.text(0.05, 0.95, info_text, fontsize=12, va='top', family='monospace')
    
    # Add legends and overall layout adjustments
    ax_main.legend(loc='upper right')
    plt.tight_layout()
    
    return fig

## Displaying Plot
Using ipywidgets to create interactive controls for adjusting the wah-wah filter parameters and seeing real-time updates.

In [6]:
# Create interactive widgets
@widgets.interact(
    f0=widgets.FloatSlider(value=1000, min=100, max=5000, step=100, 
                          description='Center Freq (Hz):', 
                          style={'description_width': 'initial'}),
    Q=widgets.FloatSlider(value=5, min=0.5, max=20, step=0.5, 
                         description='Q Factor:', 
                         style={'description_width': 'initial'}),
    filter_type=widgets.Dropdown(options=['bandpass', 'lowpass', 'highpass'], 
                               value='bandpass', 
                               description='Filter Type:', 
                               style={'description_width': 'initial'})
)
def update_plot(f0, Q, filter_type):
    """Create an interactive plot that updates based on widget values"""
    fig = create_styled_wah_plot(fs=44100, f0=f0, Q=Q, filter_type=filter_type)
    plt.close(fig)  # Prevent display of duplicate figures
    display(fig)

interactive(children=(FloatSlider(value=1000.0, description='Center Freq (Hz):', max=5000.0, min=100.0, step=1…

## Visualize Wah-Wah Pedal Sweep
Let's create a visualization showing how the filter response changes as the pedal is swept through its range.

In [7]:
# Create a wah pedal sweep visualization
plt.figure(figsize=(14, 8))
sweep_plot = plot_wah_sweep(fs=44100, f0_min=500, f0_max=2500, Q=5, n_steps=15)
plt.tight_layout()

## Bonus: 3D Visualization of Wah-Wah Effect

Let's create a 3D waterfall plot to visualize how the frequency response evolves over time as the wah parameter changes.

In [8]:
from mpl_toolkits.mplot3d import Axes3D

def create_wah_waterfall_plot(fs=44100, f0_min=500, f0_max=2500, Q=5, n_steps=30):
    """
    Create a 3D waterfall visualization of wah filter response changing over time
    """
    fig = plt.figure(figsize=(12, 9))
    ax = fig.add_subplot(111, projection='3d')
    
    # Log-spaced frequencies for evaluation (avoid zero)
    freqs = np.logspace(np.log10(20), np.log10(fs/2), 1000)
    w_normalized = 2 * np.pi * freqs / fs
    
    # Create center frequencies for the sweep
    f0_values = np.linspace(f0_min, f0_max, n_steps)
    
    # For 3D plotting
    X, Y = np.meshgrid(freqs, range(n_steps))
    Z = np.zeros_like(X)
    
    # Calculate frequency responses
    for i, f0 in enumerate(f0_values):
        b, a = create_wah_filter(fs, f0, Q, 'bandpass')
        _, h = signal.freqz(b, a, worN=w_normalized)
        # Handle potential division by zero
        h_abs = np.abs(h)
        h_abs[h_abs == 0] = np.finfo(float).eps
        db = 20 * np.log10(h_abs)
        db = np.nan_to_num(db, nan=-100, posinf=100, neginf=-100)
        Z[i, :] = db
    
    # Create a colored 3D surface
    surf = ax.plot_surface(X, Y, Z, cmap='viridis', 
                          linewidth=0, antialiased=True, alpha=0.8)
    
    # Configure the plot
    ax.set_xlabel('Frequency (Hz)')
    ax.set_ylabel('Time/Wah Position')
    ax.set_zlabel('Gain (dB)')
    ax.set_title('Wah-Wah Filter Response Over Time', fontsize=14)
    
    # Set x-axis to log scale after plotting
    ax.set_xscale('log')
    ax.set_xlim(20, fs/2)
    
    # Set y-ticks to show frequency values
    y_ticks = np.linspace(0, n_steps-1, 5, dtype=int)
    ax.set_yticks(y_ticks)
    ax.set_yticklabels([f"{f0_values[i]:.0f} Hz" for i in y_ticks])
    
    # Add color bar
    fig.colorbar(surf, ax=ax, shrink=0.5, aspect=10, label='Gain (dB)')
    
    # Adjust viewing angle
    ax.view_init(elev=30, azim=-60)
    
    plt.tight_layout()
    return fig

In [9]:
# Final interactive demo with more controls
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

@interact_manual
def interactive_wah_analysis(
    plot_type=widgets.Dropdown(options=['Frequency Response', '3D Waterfall', 'Sweep'],
                             description='Plot Type:'),
    f0=widgets.FloatSlider(value=1000, min=100, max=5000, step=50, description='Center Freq:'),
    Q=widgets.FloatSlider(value=5, min=0.5, max=20, step=0.5, description='Q Factor:'),
    filter_type=widgets.Dropdown(options=['bandpass', 'lowpass', 'highpass'], description='Filter:'),
    f0_min=widgets.FloatSlider(value=400, min=100, max=2000, step=100, description='Min Freq:'),
    f0_max=widgets.FloatSlider(value=2500, min=1000, max=5000, step=100, description='Max Freq:'),
    n_steps=widgets.IntSlider(value=20, min=5, max=40, step=5, description='Steps:')
):
    """Comprehensive interactive wah-wah filter analysis"""
    plt.figure(figsize=(14, 8))
    
    if plot_type == 'Frequency Response':
        create_styled_wah_plot(fs=44100, f0=f0, Q=Q, filter_type=filter_type)
    elif plot_type == '3D Waterfall':
        create_wah_waterfall_plot(fs=44100, f0_min=f0_min, f0_max=f0_max, Q=Q, n_steps=n_steps)
    else:  # Sweep
        plot_wah_sweep(fs=44100, f0_min=f0_min, f0_max=f0_max, Q=Q, n_steps=n_steps)
    
    plt.tight_layout()

interactive(children=(Dropdown(description='Plot Type:', options=('Frequency Response', '3D Waterfall', 'Sweep…